# 🐍 Taller de Python - Preprocesamiento de Datos
## Caso Olist E-commerce

### Contexto del Negocio

Continuamos trabajando como **Analistas de Datos** en Olist. Ahora que exploramos los datos con SQL, necesitamos **preprocesarlos** para poder construir modelos predictivos y dashboards.

### Objetivos del Taller
1. Cargar y explorar datos con Pandas
2. Identificar y manejar valores nulos
3. Detectar y tratar valores atípicos
4. Transformar y crear nuevas variables
5. Integrar datos de múltiples fuentes

---

## ⚙️ Configuración Inicial

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente")

In [ ]:
# Cargar los datasets
# Asegúrate de que los archivos estén en la carpeta ./data/

data_path = './data/'

# Cargar tablas principales
df_customers = pd.read_csv(f'{data_path}olist_customers_dataset.csv')
df_orders = pd.read_csv(f'{data_path}olist_orders_dataset.csv')
df_items = pd.read_csv(f'{data_path}olist_order_items_dataset.csv')
df_payments = pd.read_csv(f'{data_path}olist_order_payments_dataset.csv')
df_reviews = pd.read_csv(f'{data_path}olist_order_reviews_dataset.csv')
df_products = pd.read_csv(f'{data_path}olist_products_dataset.csv')
df_sellers = pd.read_csv(f'{data_path}olist_sellers_dataset.csv')
df_categories = pd.read_csv(f'{data_path}product_category_name_translation.csv')

print("✅ Datasets cargados:")
print(f"   - Clientes: {len(df_customers):,} registros")
print(f"   - Pedidos: {len(df_orders):,} registros")
print(f"   - Items: {len(df_items):,} registros")
print(f"   - Pagos: {len(df_payments):,} registros")
print(f"   - Reviews: {len(df_reviews):,} registros")
print(f"   - Productos: {len(df_products):,} registros")
print(f"   - Vendedores: {len(df_sellers):,} registros")

---

## 🔹 PARTE 1: Exploración de Datos (EDA Básico)

### Objetivo
Antes de preprocesar, necesitamos **entender** nuestros datos: estructura, tipos, distribuciones.

### Ejercicio 1.1 - Estructura del DataFrame

**Pregunta de negocio:** ¿Qué información tenemos de los pedidos?

Explora el DataFrame `df_orders`:

In [ ]:
# Mostrar las primeras filas
df_orders.head()

In [ ]:
# Ver información del DataFrame
df_orders.info()

In [ ]:
# Estadísticas descriptivas
df_orders.describe()

### Ejercicio 1.2 - Explorar la tabla de productos

**Tu turno:** Explora el DataFrame `df_products` usando `.head()`, `.info()` y `.describe()`

In [ ]:
# Tu código aquí


### Ejercicio 1.3 - Distribución de precios

**Pregunta de negocio:** ¿Cómo se distribuyen los precios de los productos vendidos?

Crea un histograma de la columna `price` en `df_items`:

In [ ]:
# Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```python
fig, ax = plt.subplots(figsize=(10, 5))
df_items['price'].hist(bins=50, ax=ax, edgecolor='white')
ax.set_xlabel('Precio (R$)')
ax.set_ylabel('Frecuencia')
ax.set_title('Distribución de Precios de Productos')
plt.show()

print(f"\nEstadísticas de precio:")
print(df_items['price'].describe())
```
</details>

---

## 🔹 PARTE 2: Manejo de Valores Nulos

### Objetivo
Los valores nulos pueden afectar nuestros análisis. Necesitamos identificarlos y decidir cómo tratarlos.

### Ejercicio 2.1 - Identificar valores nulos

**Pregunta de negocio:** ¿Qué tan completos están nuestros datos de productos?

In [ ]:
# Contar valores nulos por columna
def reporte_nulos(df, nombre_df):
    """Genera un reporte de valores nulos"""
    nulos = df.isnull().sum()
    porcentaje = (nulos / len(df)) * 100
    
    reporte = pd.DataFrame({
        'Columna': nulos.index,
        'Nulos': nulos.values,
        'Porcentaje': porcentaje.values
    })
    reporte = reporte[reporte['Nulos'] > 0].sort_values('Nulos', ascending=False)
    
    print(f"\n📊 Reporte de Nulos - {nombre_df}")
    print(f"Total de registros: {len(df):,}")
    print("-" * 50)
    
    if len(reporte) == 0:
        print("✅ No hay valores nulos!")
    else:
        display(reporte)
    
    return reporte

# Aplicar a productos
reporte_nulos(df_products, 'Productos');

### Ejercicio 2.2 - Revisar nulos en otras tablas

**Tu turno:** Genera el reporte de nulos para `df_orders` y `df_reviews`

In [ ]:
# Tu código aquí


### Ejercicio 2.3 - Tratar valores nulos en productos

**Decisión de negocio:** 
- Para `product_category_name`: reemplazar con "sin_categoria"
- Para medidas del producto (peso, dimensiones): reemplazar con la mediana

**Tu turno:** Implementa estas transformaciones

In [ ]:
# Crear copia para no modificar el original
df_products_clean = df_products.copy()

# Tu código aquí: rellenar nulos


<details>
<summary>💡 Ver solución</summary>

```python
# Crear copia
df_products_clean = df_products.copy()

# Reemplazar categoría nula
df_products_clean['product_category_name'].fillna('sin_categoria', inplace=True)

# Reemplazar medidas con la mediana
columnas_medidas = ['product_weight_g', 'product_length_cm', 
                    'product_height_cm', 'product_width_cm']

for col in columnas_medidas:
    mediana = df_products_clean[col].median()
    df_products_clean[col].fillna(mediana, inplace=True)
    print(f"✓ {col}: nulos reemplazados con mediana = {mediana:.2f}")

# Verificar
print(f"\n✅ Nulos restantes: {df_products_clean.isnull().sum().sum()}")
```
</details>

---

## 🔹 PARTE 3: Detección y Tratamiento de Outliers

### Objetivo
Los valores atípicos pueden distorsionar nuestros análisis. Necesitamos identificarlos y decidir qué hacer con ellos.

### Ejercicio 3.1 - Visualizar outliers con boxplot

**Pregunta de negocio:** ¿Hay precios atípicos en nuestras ventas?

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot de precio
axes[0].boxplot(df_items['price'])
axes[0].set_title('Distribución de Precios')
axes[0].set_ylabel('Precio (R$)')

# Boxplot de freight (costo de envío)
axes[1].boxplot(df_items['freight_value'])
axes[1].set_title('Distribución de Costos de Envío')
axes[1].set_ylabel('Costo de Envío (R$)')

plt.tight_layout()
plt.show()

### Ejercicio 3.2 - Método IQR para detectar outliers

**Tu turno:** Implementa una función que detecte outliers usando el método IQR (Rango Intercuartílico)

In [ ]:
def detectar_outliers_iqr(df, columna):
    """
    Detecta outliers usando el método IQR
    Retorna los límites y cantidad de outliers
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    # Tu código aquí: calcular límites y contar outliers
    limite_inferior = None  # Completar
    limite_superior = None  # Completar
    
    # Contar outliers
    outliers = None  # Completar: filtrar los outliers
    
    return {
        'columna': columna,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'limite_inferior': limite_inferior,
        'limite_superior': limite_superior,
        'n_outliers': len(outliers) if outliers is not None else 0,
        'porcentaje': (len(outliers) / len(df) * 100) if outliers is not None else 0
    }

# Probar con precio
# detectar_outliers_iqr(df_items, 'price')

<details>
<summary>💡 Ver solución</summary>

```python
def detectar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    
    return {
        'columna': columna,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'limite_inferior': limite_inferior,
        'limite_superior': limite_superior,
        'n_outliers': len(outliers),
        'porcentaje': (len(outliers) / len(df) * 100)
    }

resultado = detectar_outliers_iqr(df_items, 'price')
for k, v in resultado.items():
    print(f"{k}: {v:.2f}" if isinstance(v, float) else f"{k}: {v}")
```
</details>

### Ejercicio 3.3 - Tratar outliers (opcional)

**Decisión de negocio:** Para algunos análisis, vamos a "capear" (capping) los outliers al percentil 99.

Esto significa que los valores extremos se reemplazan por el valor del percentil 99.

In [ ]:
def capear_outliers(df, columna, percentil_inferior=1, percentil_superior=99):
    """Capea los outliers a los percentiles especificados"""
    df_nuevo = df.copy()
    
    p_inf = df[columna].quantile(percentil_inferior/100)
    p_sup = df[columna].quantile(percentil_superior/100)
    
    df_nuevo[columna] = df_nuevo[columna].clip(lower=p_inf, upper=p_sup)
    
    print(f"Columna: {columna}")
    print(f"  Antes  -> Min: {df[columna].min():.2f}, Max: {df[columna].max():.2f}")
    print(f"  Después -> Min: {df_nuevo[columna].min():.2f}, Max: {df_nuevo[columna].max():.2f}")
    
    return df_nuevo

# Aplicar a precio
df_items_clean = capear_outliers(df_items, 'price')

---

## 🔹 PARTE 4: Transformación de Datos

### Objetivo
Crear nuevas variables y transformar las existentes para facilitar el análisis.

### Ejercicio 4.1 - Convertir fechas

**Pregunta de negocio:** Las fechas están como texto, necesitamos convertirlas para hacer análisis temporales.

In [ ]:
# Ver el tipo actual
print("Tipo de dato actual:")
print(df_orders['order_purchase_timestamp'].dtype)
print("\nEjemplo de valores:")
print(df_orders['order_purchase_timestamp'].head())

In [ ]:
# Convertir a datetime
df_orders_clean = df_orders.copy()

columnas_fecha = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in columnas_fecha:
    df_orders_clean[col] = pd.to_datetime(df_orders_clean[col])

print("✅ Fechas convertidas")
df_orders_clean.dtypes

### Ejercicio 4.2 - Extraer componentes de fecha

**Tu turno:** Crea nuevas columnas para:
- Año de compra
- Mes de compra
- Día de la semana
- Hora de compra

In [ ]:
# Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```python
# Extraer componentes de la fecha de compra
df_orders_clean['año_compra'] = df_orders_clean['order_purchase_timestamp'].dt.year
df_orders_clean['mes_compra'] = df_orders_clean['order_purchase_timestamp'].dt.month
df_orders_clean['dia_semana'] = df_orders_clean['order_purchase_timestamp'].dt.dayofweek
df_orders_clean['hora_compra'] = df_orders_clean['order_purchase_timestamp'].dt.hour

# Nombres de días en español
dias_semana = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 
               3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
df_orders_clean['nombre_dia'] = df_orders_clean['dia_semana'].map(dias_semana)

print("✅ Columnas de fecha creadas")
df_orders_clean[['order_purchase_timestamp', 'año_compra', 'mes_compra', 
                 'nombre_dia', 'hora_compra']].head()
```
</details>

### Ejercicio 4.3 - Calcular tiempo de entrega

**Pregunta de negocio:** ¿Cuántos días tarda en promedio la entrega de un pedido?

In [ ]:
# Calcular días de entrega (solo para pedidos entregados)
df_entregados = df_orders_clean[df_orders_clean['order_status'] == 'delivered'].copy()

df_entregados['dias_entrega'] = (
    df_entregados['order_delivered_customer_date'] - 
    df_entregados['order_purchase_timestamp']
).dt.days

print(f"Tiempo promedio de entrega: {df_entregados['dias_entrega'].mean():.1f} días")
print(f"Mediana: {df_entregados['dias_entrega'].median():.1f} días")
print(f"Mínimo: {df_entregados['dias_entrega'].min()} días")
print(f"Máximo: {df_entregados['dias_entrega'].max()} días")

### Ejercicio 4.4 - Crear variable de entrega tardía

**Tu turno:** Crea una columna `entrega_tardia` que sea:
- 1 si la entrega fue después de la fecha estimada
- 0 si fue a tiempo o antes

In [ ]:
# Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```python
df_entregados['entrega_tardia'] = (
    df_entregados['order_delivered_customer_date'] > 
    df_entregados['order_estimated_delivery_date']
).astype(int)

print("Distribución de entregas:")
print(df_entregados['entrega_tardia'].value_counts(normalize=True).map('{:.2%}'.format))
```
</details>

---

## 🔹 PARTE 5: Integración de Datos (Merge)

### Objetivo
Combinar información de múltiples tablas para crear un dataset consolidado.

### Ejercicio 5.1 - Unir pedidos con clientes

**Pregunta de negocio:** Necesitamos saber de qué estados provienen los pedidos.

In [ ]:
# Merge de pedidos con clientes
df_completo = df_orders_clean.merge(
    df_customers[['customer_id', 'customer_city', 'customer_state']], 
    on='customer_id', 
    how='left'
)

print(f"Registros resultantes: {len(df_completo):,}")
df_completo.head()

### Ejercicio 5.2 - Agregar información de items y pagos

**Tu turno:** Calcula el valor total de cada pedido uniendo con `df_items` y agrupando

In [ ]:
# Tu código aquí: calcular el total por pedido


<details>
<summary>💡 Ver solución</summary>

```python
# Agrupar items por pedido
df_totales_pedido = df_items.groupby('order_id').agg({
    'price': 'sum',
    'freight_value': 'sum',
    'order_item_id': 'count'  # número de items
}).reset_index()

df_totales_pedido.columns = ['order_id', 'total_precio', 'total_envio', 'num_items']
df_totales_pedido['total_pedido'] = df_totales_pedido['total_precio'] + df_totales_pedido['total_envio']

# Unir con el dataframe completo
df_completo = df_completo.merge(df_totales_pedido, on='order_id', how='left')

print(f"Columnas actuales: {df_completo.columns.tolist()}")
df_completo[['order_id', 'customer_state', 'total_precio', 'total_envio', 'total_pedido']].head()
```
</details>

### Ejercicio 5.3 - Agregar reviews

**Tu turno:** Une el `review_score` de la tabla `df_reviews` al dataset completo

In [ ]:
# Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```python
# Unir reviews (solo score)
df_completo = df_completo.merge(
    df_reviews[['order_id', 'review_score']], 
    on='order_id', 
    how='left'
)

print(f"Pedidos con review: {df_completo['review_score'].notna().sum():,}")
print(f"Pedidos sin review: {df_completo['review_score'].isna().sum():,}")
```
</details>

---

## 🎯 DESAFÍO FINAL

### Crear Dataset Analítico

El equipo de ciencia de datos necesita un dataset limpio y consolidado para construir un modelo de predicción de satisfacción del cliente.

**Requerimientos:**
1. Solo incluir pedidos entregados
2. Eliminar pedidos sin review
3. Crear variable target: `cliente_satisfecho` (1 si score >= 4, 0 si no)
4. Incluir: estado del cliente, total del pedido, días de entrega, si fue tardío, número de items
5. Guardar como CSV

In [ ]:
# DESAFÍO: Tu código aquí


---

## 📝 Resumen de Conceptos Python/Pandas

| Concepto | Ejemplo | Uso |
|----------|---------|-----|
| `.head()` | `df.head()` | Ver primeras filas |
| `.info()` | `df.info()` | Estructura del DataFrame |
| `.isnull()` | `df.isnull().sum()` | Detectar nulos |
| `.fillna()` | `df['col'].fillna(0)` | Rellenar nulos |
| `.describe()` | `df.describe()` | Estadísticas descriptivas |
| `.groupby()` | `df.groupby('col').sum()` | Agrupar y agregar |
| `.merge()` | `df1.merge(df2, on='id')` | Unir DataFrames |
| `pd.to_datetime()` | `pd.to_datetime(df['fecha'])` | Convertir a fecha |
| `.dt.year` | `df['fecha'].dt.year` | Extraer año |
| `.quantile()` | `df['col'].quantile(0.25)` | Calcular percentiles |
| `.clip()` | `df['col'].clip(0, 100)` | Limitar valores |

---

**¡Felicitaciones!** Has completado el taller de preprocesamiento de datos. 🎉